In [1]:
import pandas as pd
import datetime as dt

In [2]:
ema = pd.read_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/ema_mio202537.csv")

In [6]:
ema = ema.copy()

# Calcular venta_costo_ema
ema["venta_costo_ema"] = (ema["EMA"] * ema["producto_costo_unitario"]).round(2)

# Totales
total_costo = ema["venta_costo_ema"].sum()
total_ema = ema["EMA"].sum()

# Proporciones
ema["venta_costo"] = (ema["venta_costo_ema"] / total_costo).round(4)   # %
#ema["cantidad"] = (ema["EMA"] / total_ema).round(4)                    # %

# Ordenar por venta_costo descendente
ema = ema.sort_values("venta_costo", ascending=False).reset_index(drop=True)

# Acumulados en %
ema["acumulado_costo"] = ema["venta_costo"].cumsum().round(4)
#ema["acumulado_cantidad"] = ema["cantidad"].cumsum().round(4)


# 6. (opcional) Renombrar columnas para que se vean como en tu reporte Odoo
ema_def = ema.rename(columns={
    "año": "Año",
    "semana_num": "Semana",
    "EMA": "Ema",
    "producto_costo_unitario": "Costo unitario",
    "venta_costo_ema" : "Venta costo EMA",
    "venta_costo":"Venta costo%",
    "acumulado_costo" :"Acumulado costo",
    "desviacion_ema%":"Desviacion EMA %"
})



ema_def["Desviacion EMA"] = (ema_def["Venta costo EMA"] * ema_def["Desviacion EMA %"]).round(2)
ema_def=ema_def.drop(columns=["Unnamed: 0"])

In [7]:
ema_def[ema_def["store_name"]=="SUCURSAL BARRANQUILLA"].sort_values(by=["Ema"],ascending=False).head(10)

,store_name,product_ref,Año,Semana,num_facturas,Ema,demanda_ajustada,Costo unitario,Desviacion EMA %,coef_ventas,Venta costo EMA,Venta costo%,Acumulado costo,Desviacion EMA
38,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,37,7.0,38.554926,10.0,13507.71,0.193115,3.935484,520788.76,0.0027,0.2061,100572.21
6,SUCURSAL BARRANQUILLA,BLS00037125,2025.0,37,9.0,33.981446,22.0,32797.97,0.112285,1.585421,1114522.45,0.0058,0.0962,125144.68
125,SUCURSAL BARRANQUILLA,DAB14570025,2025.0,37,6.0,30.869047,22.0,9774.98,0.626906,5.621262,301744.32,0.0016,0.3816,189165.31
107,SUCURSAL BARRANQUILLA,DAB02772025,2025.0,37,4.0,30.471810,7.0,10680.97,0.170460,3.096774,325468.49,0.0017,0.3519,55479.31
128,SUCURSAL BARRANQUILLA,DAB02666025,2025.0,37,6.0,25.581851,26.0,11371.65,0.107747,2.157303,290907.86,0.0015,0.3863,31344.38
45,SUCURSAL BARRANQUILLA,BCS00035125,2025.0,37,11.0,23.910288,15.0,20638.72,0.158093,2.307692,493477.74,0.0025,0.2240,78015.60
47,SUCURSAL BARRANQUILLA,BCS10035125,2025.0,37,7.0,22.682642,11.0,20956.60,0.135993,2.476190,475351.06,0.0025,0.2290,64644.19
27,SUCURSAL BARRANQUILLA,BCS00025125,2025.0,37,5.0,19.326151,6.0,31058.67,0.081797,1.483636,600244.55,0.0031,0.1758,49098.43
34,SUCURSAL BARRANQUILLA,BCS00249125,2025.0,37,8.0,19.151567,12.0,27520.25,0.112979,1.811321,527055.91,0.0027,0.1953,59546.27
8,SUCURSAL BARRANQUILLA,DAB02982025,2025.0,37,3.0,18.685080,10.0,48785.61,0.210170,3.172414,911563.03,0.0047,0.1059,191583.05
